In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re

# to avoid warnings
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

sns.set(style='darkgrid')
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tdf = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')
tdf.head()

In [ ]:

tdf= tdf[['Location', 'MinTemp', 'MaxTemp', 'Rainfall','WindGustDir', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 
      'Pressure9am', 'Pressure3pm','Humidity9am','Humidity3pm']]
tdf.head()

In [ ]:
tdf.info()

In [ ]:
tdf.shape

In [ ]:
tdf.isnull().sum()

In [ ]:
import missingno as msno
msno.matrix(tdf);

In [ ]:
msno.bar(tdf);

In [ ]:
tdf.dropna(inplace=True)

In [ ]:
msno.bar(tdf);

In [ ]:
tdf.describe().T

In [ ]:
tdf['Location'].unique()

In [ ]:
len(tdf['Location'].unique())

In [ ]:
len(tdf['WindGustDir'].unique())

In [ ]:
tdf.shape

In [ ]:
tdf.dtypes

In [ ]:
sp = tdf['Location'].value_counts()
sp = pd.DataFrame(sp)
sp.T

In [ ]:
tdf.corr()

In [ ]:
sns.heatmap(tdf.corr(), annot=True, cmap='YlGnBu');

In [ ]:
mask = np.zeros_like(tdf.corr())
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(tdf.corr(), annot=True, mask=mask, cmap='Reds')

In [ ]:
dmy = pd.get_dummies(tdf, drop_first=True)
X = dmy.drop(['MaxTemp'], axis=1)
y = dmy['MaxTemp']

# ML Modelling
# Data setimizi Train ve Test olarak Bölelim

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42 )

In [ ]:
print (' Test ve Train veri setlerindeki gözlem sayıları...')
print('X_train: ', np.shape(X_train))
print('y_train: ', np.shape(y_train))
print('X_test: ', np.shape(X_test))
print('y_test: ', np.shape(y_test))

In [ ]:
reg = LinearRegression()
model = reg.fit(X_train, y_train)

In [ ]:
print('Model intercept: ', reg.intercept_)
print('Model coefficients: ', reg.coef_)

In [ ]:
Train=model.score(X_train, y_train)
Train

In [ ]:
Test = model.score(X_test, y_test)
Test

In [ ]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

In [ ]:
cross_val_score(model, X_train, y_train, cv = 10, scoring = 'r2')

In [ ]:
Cross_Val= cross_val_score(model, X_train, y_train, cv = 10, scoring = 'r2').mean()
Cross_Val

Model R2 value (0,8643) and cross validation R2 value (0,8642) are almost identical, we can conclude that there is no overfitting.

In [ ]:
Score=  [Train, Test, Cross_Val]
dic = {'R2_Score':Score}

pd.DataFrame(dic).T.rename(columns={ 0:'Train',1:'Test', 2:'Cross_Val'})

 # Ridge regresyon modeli

In [ ]:
ridge_model = Ridge().fit(X_train, y_train)

In [ ]:
ridge_model

In [ ]:
ridge_model.score(X_train, y_train) , ridge_model.score(X_test, y_test)

In [ ]:
np.sqrt(mean_squared_error(y_train, ridge_model.predict(X_train))) , np.sqrt(mean_squared_error(y_test, ridge_model.predict(X_test)))

In [ ]:
ridge_model = Ridge(alpha=10000).fit(X_train, y_train)
ridge_model

In [ ]:
ridge_model.score(X_train, y_train) , ridge_model.score(X_test, y_test)

In [ ]:
np.sqrt(mean_squared_error(y_train, ridge_model.predict(X_train))) , np.sqrt(mean_squared_error(y_test, ridge_model.predict(X_test)))

In [ ]:
np.linspace(100,0.0001,100)

In [ ]:
lambdalar = np.linspace(100,0.0001,100)
ridge_model = Ridge()
katsayilar = []

for i in lambdalar:
    ridge_model.set_params(alpha = i)
    ridge_model.fit(X_train, y_train)
    katsayilar.append(ridge_model.coef_)
    
ax=plt.gca()
ax.plot(lambdalar, katsayilar)
ax.set_xscale('log')

plt.xlabel('Lambda(Alpha) Değerleri')
plt.ylabel('Katsayılar/Ağırlıklar')
plt.title('Ridge Katsayıları')

In [ ]:
alphaCV = RidgeCV(alphas = lambdalar, 
                  scoring = 'neg_mean_squared_error',
                  normalize = True)

In [ ]:
alphaCV.fit(X_train, y_train)

In [ ]:
alphaCV.alpha_

In [ ]:
alphaCV.coef_

In [ ]:
ridge_tuned = Ridge(alpha= alphaCV.alpha_,
                   normalize = True).fit(X_train,y_train)

In [ ]:
Train_R=ridge_tuned.score(X_train,y_train)
Train_R

In [ ]:
Test_R=ridge_tuned.score(X_test,y_test)
Test_R

In [ ]:
Cross_Val_R= cross_val_score(ridge_model, X_train, y_train, cv = 10, scoring = 'r2').mean()
Cross_Val_R

In [ ]:
ridge_tuned.score(X_test,y_test), np.sqrt(mean_squared_error(y_test, ridge_tuned.predict(X_test)))

In [ ]:
Score_R=  [Train_R, Test_R, Cross_Val_R]
dic_R = {'R2_Score':Score_R}

pd.DataFrame(dic_R).T.rename(columns={ 0:'Train',1:'Test', 2:'Cross_Val'})

# Lasso Regresyon modeli

In [ ]:
lasso_model = Lasso().fit(X_train, y_train)

In [ ]:
lasso_model

In [ ]:
lasso_model.score(X_train, y_train) , lasso_model.score(X_test, y_test)

In [ ]:
np.sqrt(mean_squared_error(y_train, lasso_model.predict(X_train))) , np.sqrt(mean_squared_error(y_test, lasso_model.predict(X_test)))

In [ ]:
np.sum(lasso_model.coef_!=0)

In [ ]:
lasso_model.coef_

In [ ]:
lasso_model_cv = LassoCV(alphas = None, cv = 10, max_iter =1000, normalize = True)

In [ ]:
lasso_model_cv.fit(X_train, y_train)

In [ ]:
lasso_model_cv.alpha_

In [ ]:
lasso_optimum = Lasso(alpha= lasso_model_cv.alpha_) 

In [ ]:
lasso_optimum.fit(X_train, y_train)

In [ ]:
Train_L=lasso_optimum.score(X_train, y_train) 
Train_L

In [ ]:
Test_L=lasso_optimum.score(X_test, y_test)
Test_L

In [ ]:
Cross_Val_L= cross_val_score(lasso_model, X_train, y_train, cv = 10, scoring = 'r2').mean()
Cross_Val_L

In [ ]:
np.sqrt(mean_squared_error(y_train, lasso_model_cv.predict(X_train))) , np.sqrt(mean_squared_error(y_test, lasso_model_cv.predict(X_test)))

In [ ]:
Score_L=  [Train_L, Test_L, Cross_Val_L]
dic_L = {'R2_Score':Score_L}

pd.DataFrame(dic_L).T.rename(columns={ 0:'Train',1:'Test', 2:'Cross_Val'})

# ElasticNet regresyonu

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
elas_model = ElasticNet().fit(X_train, y_train)

In [ ]:
elas_model.coef_ , elas_model.intercept_

In [ ]:
elas_model.score(X_train, y_train) , elas_model.score(X_test, y_test) 

In [ ]:
y_tahmin = elas_model.predict(X_test)

In [ ]:
y_tahmin

In [ ]:
np.sqrt(mean_squared_error(y_test, y_tahmin)) 

In [ ]:
np.sqrt(mean_squared_error(y_train, elas_model.predict(X_train))), np.sqrt(mean_squared_error(y_test, elas_model.predict(X_test)))

In [ ]:
 elas_model_cv= ElasticNetCV(cv=10, random_state = 42).fit(X_train,y_train)

In [ ]:
elas_model_cv.alpha_

In [ ]:
elas_model_cv.coef_

In [ ]:
elas_optimum = ElasticNet(alpha = elas_model_cv.alpha_, l1_ratio=0.5).fit(X_train, y_train)
elas_optimum

In [ ]:
Train_E= elas_optimum.score(X_train, y_train) 
Test_E=elas_optimum.score(X_test, y_test)

In [ ]:
Train_E

In [ ]:
Test_E

In [ ]:
np.sqrt(mean_squared_error(y_train, elas_optimum.predict(X_train))) , np.sqrt(mean_squared_error(y_test, elas_optimum.predict(X_test)))

In [ ]:
Cross_Val_E= cross_val_score(elas_model, X_train, y_train, cv = 10, scoring = 'r2').mean()
Cross_Val_E

In [ ]:
Score_E=  [Train_E, Test_E, Cross_Val_E]
dic_E = {'R2_Score':Score_E}

pd.DataFrame(dic_E).T.rename(columns={ 0:'Train',1:'Test', 2:'Cross_Val'})

# Polinom Regresyon


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(2)
X_train = poly.fit_transform(X_train)

In [ ]:
poly_model = LinearRegression()
poly_model.fit(X_train, y_train)

In [ ]:
poly_model.score(X_train, y_train)

In [ ]:
X_test = poly.fit_transform(X_test)

In [ ]:
poly_model.score(X_test, y_test)

In [ ]:
poly_model.score(X_train, y_train)

In [ ]:
np.sqrt(mean_squared_error(y_train,poly_model.predict(X_train))) , np.sqrt(mean_squared_error(y_test,poly_model.predict(X_test)))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(poly_model, X_train, y_train, cv = 10, scoring = 'r2').mean()